### Parte 1

Importar las bibliotecas que son usadas en el desarrollo del laboratorio

In [56]:
#Bibliotecas propias
%matplotlib inline
import pandas as pd
import os,os.path,math,scipy,cv2
import PIL

from datasets import ClassLabel

#Bibliotecas para LeNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

%matplotlib inline
import os
import torch
import torchvision
from torch import nn
from d2l import torch as d2l


In [3]:
#Se setean las seeds
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
### Escribir en estos bloques el codigo de acorde a las instrucciones planteadas
### pero no olvide que puede crear los bloques que quiera y puede agregar las sub secciones 
### que deseen, esto es solo una guía basica

Lectura del conjunto de datos, deberá diseñar una estrategia para la selección y lectura del sub conjunto de datos, es decir, se debe crear un conjuntos de datos pequeño que contenga entre 3 a 5 clases y una cantidad de $N$ ejemplos para cada clase, con el fin de tener un conjunto de datos balanceado con un tamaño que se pueda trabajar con los recursos disponibles.

In [5]:
def same_amount_of_n_images(img_dir_list, N):
    for img_dir in img_dir_list:
        img_count = len([name for name in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, name))])
        if(img_count < N):
            return False
    return True

In [6]:
def storage_images(data_frame ,img_label,img_dir):
    count = 0
    trans = transforms.Compose([transforms.ToTensor()])
    for img in os.listdir(img_dir):
        if(count < N):
            path = os.path.join(img_dir,img)
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            img = cv2.resize(img,(TAM_MAX,TAM_MAX))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #img = PIL.Image.open(path)
            #img = img.resize((TAM_MAX,TAM_MAX))
            img = trans(img)/255
            #img = img[:3]
            
            data_frame.append((img,img_label))
            count+=1
        

In [38]:
#Direccion para carpetas de clases
chihuahua_dir = 'images/n02085620-Chihuahua'
japanese_spaniel_dir = 'images/n02085782-Japanese_spaniel'
maltese_dir = 'images/n02085936-Maltese_dog'
pekinese_dir = 'images/n02086079-Pekinese'
toy_terrier_dir = 'images/n02087046-toy_terrier'
aux_dir_list = [chihuahua_dir,japanese_spaniel_dir,maltese_dir,pekinese_dir,toy_terrier_dir]

#Se define la cantidad N de ejemplos por clase
N = 140

#Se define el tamaño maximo de imagen
TAM_MAX = 224

#Data frame para almacenar las tuplas (imagen,label)
data_frame = []

#Se modifica el nombre de los labels
labels_names = ['chihuahua','japanese_spaniel','maltese','pekinese','toy_terrier']

#Se verifica que todos los directorios tengan la misma cantidad N de imagenes.
if(same_amount_of_n_images(aux_dir_list,N)):
    #Se almacenan las imagenes, y los labels
    storage_images(data_frame,1,chihuahua_dir)
    storage_images(data_frame,2,japanese_spaniel_dir)
    storage_images(data_frame,3,maltese_dir)
    storage_images(data_frame,4,pekinese_dir)
    storage_images(data_frame,5,toy_terrier_dir)
else:
    print('N invalido')


Realizar la división de los datos en dos conjuntos de datos, Train (70%) y Test (30%).

In [39]:
train_ratio = 0.7
valid_ratio = 0.1

n_train_examples = int(len(data_frame) * train_ratio)
n_test_examples = len(data_frame) - n_train_examples
n_validate_examples = int(n_train_examples*valid_ratio)


train_data, test_data = data.random_split(data_frame, 
                                           [n_train_examples, n_test_examples])
train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples-n_validate_examples, n_validate_examples])

Aplicar técnicas para aumentar la cantidad de imágenes en los datos de entrenamiento, recortando las imágenes, girando, modificando los contrastes, entre otras. Como mínimo, se debe utilizar dos técnicas.

In [40]:
#Tecnica 1: Rotar imagen

#Tecnica 2: Recortar imagen

### Parte 2

Creación de una arquitectura desde cero, para esté paso se recomienda adaptar la implementación LeNet vista en clases. Cuya implementación debe funcionar tanto en CPU como en GPU.

In [41]:
#Se crean los dataloaders para procesar los datos a travez de las iteraciones.
#Se trabaja con un tamaño de lote BATCH_SIZE
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

test_iterator = data.DataLoader(test_data, 
                                 batch_size = BATCH_SIZE)


In [43]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 6, 
                               out_channels = 16, 
                               kernel_size = 5)
        self.fc_1 = nn.Linear(44944, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, output_dim)

    def forward(self, x):
        #x = [batch size, 1, 28, 28]
        x = self.conv1(x)
        #x = [batch size, 6, 24, 24]
        x = F.max_pool2d(x, kernel_size = 2)
        #x = [batch size, 6, 12, 12]
        x = F.relu(x)
        x = self.conv2(x)
        #x = [batch size, 16, 8, 8]
        x = F.max_pool2d(x, kernel_size = 2)
        #x = [batch size, 16, 4, 4]
        x = F.relu(x)
        #print(x.shape[0]) 
        #torch.Size([64, 16, 4, 4]) 64 vectore de cada uno 256 pixeles [16 + 16 + 16 + 16 .... + 16]
        x = x.view(x.shape[0], -1)
        #print(x.shape) 
        #torch.Size([64, 256])
        #x = [batch size, 16*4*4 = 256]
        h = x
        x = self.fc_1(x)
        
        #x = [batch size, 120]
        
        x = F.relu(x)

        x = self.fc_2(x)
        
        #x = batch size, 84]
        
        x = F.relu(x)

        x = self.fc_3(x)

        #x = [batch size, output dim]
        
        return x, h

In [44]:
OUTPUT_DIM = 10

model = LeNet(OUTPUT_DIM)

In [45]:
optimizer = optim.Adam(model.parameters())

In [46]:
criterion = nn.CrossEntropyLoss()

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [48]:
device

device(type='cpu')

In [49]:
model = model.to(device)
criterion = criterion.to(device)

In [50]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [51]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = torch.tensor(y).to(device)
        #y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = torch.tensor(y).to(device)
            #y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

<ipython-input-51-5d08398568d7>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(device)
<ipython-input-52-ec60f0ed50f8>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(device)


Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 1.975 | Train Acc: 21.14%
	 Val. Loss: 1.693 |  Val. Acc: 12.50%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 1.647 | Train Acc: 19.00%
	 Val. Loss: 1.676 |  Val. Acc: 12.50%


In [55]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<ipython-input-52-ec60f0ed50f8>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(device)


Test Loss: 1.650 | Test Acc: 15.81%


Probar el modelo por lo menos con tres configuraciones de parámetros, entre los que se puede variar número de convoluciones, función de activación, número de neuronas en capas densas, entre otros.

Crear los gráficos de entrenamiento, donde se vea el error en los conjuntos de train y test durante cada época de entrenamiento.

Crear matriz de clasificación del modelo que presento el menor error en la clasificación, por lo tanto la mejor clasificación.

### Parte 3

Importar una arquitectura profunda que se encuentra pre-entrenada con ImageNet

In [60]:
finetune_net = torchvision.models.resnet18(pretrained=True)

Realizar la modificación en la arquitectura para poder clasificar el número de clases que usted haya escogido.

In [61]:
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 5)
nn.init.xavier_uniform_(finetune_net.fc.weight);

Aplicar Fine-tuning en la arquitectura profunda, para re-entrenar con el conjunto de datos seleccionado manteniendo los pesos de las arquitecturas ResNet, VGG o DenseNet. Cuya implementación creada debe funcionar tanto en CPU como en GPU.

Probar el modelo por lo menos con tres configuraciones de parámetros.

Crear los gráficos de entrenamiento, donde se vea el error en los conjuntos de train y test durante cada época de entrenamiento.

Crear matriz de clasificación del modelo que presento el menor error en la clasificación, por lo tanto la mejor clasificación.